In [1]:
# Quietly compile autovarlamba.f90
import os
import sys
# os.system("f2py -c -m autovarlambda SR.read_write_as.for f2py_autovarlambda.f90 --quiet")
# print("\n===>>> autovarlambda compiled with f2py <<<===")

# Import other stuff
import numpy as np
import autovarlambda
import time
import pandas as pd
import yaml
import GPy
import GPyOpt
from numpy.random import seed
seed(123456)


import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib import gridspec

colors=['k','r','g','b','m','y','c','tab:blue',
        'tab:orange','tab:green','tab:red','tab:purple','tab:brown','tab:pink','tab:gray']
orb=['1s','2s','2p','3s','3p','3d','4s','4p','4d','4f','5s','5p','5d','5f','5g']
symbs=['o','s','v','^','>','<','o','o','o','o','o','o','o','o','o']

# Input parameters
global gridtype, cfgs, maxevals, mtype, aftype, afweight, gridtype, igrid, initer, ipol
global nlamvar, maxlam, minlam, mfunc, ifun, minst, imin, nener, wi, igi

print("\n===>>> Input parameters <<<===\n")
with open("varlam_gpyopt.yml", 'r') as stream:
    data_loaded=yaml.load(stream)

cfgs=data_loaded.get("cfgs")
if cfgs is None: raise ValueError("Error: cfgs not defined.")

initer=data_loaded.get("initer")
if initer is None: raise ValueError("Error: initer not defined.")
print(" * initial evaluations: "+str(initer))

maxevals=data_loaded.get("maxevals")
if maxevals is None: raise ValueError("Error: maxevals not defined.")
print(" * evaluations: "+str(maxevals))

mtype=data_loaded.get("mtype")
if mtype=="GP": print(" * model type: "+mtype)
elif mtype is None: 
    mtype="GP"
    print("Warning: model type not defined. Use default: GP")
else: raise ValueError("Error: "+mtype+" not implemented.")

aftype=data_loaded.get("aftype")
if aftype=="EI" or aftype=="MPI" or aftype=="LCB": print(" * acquisition function type: "+aftype)
elif aftype is None: 
    aftype="EI"
    print("Warning: acquisition function type not defined. Use default: EI")
else: raise ValueError("Error: "+mtype+" not implemented.")

afweight=data_loaded.get("afweight")
if afweight: print(" * acquisition function weight: "+str(afweight))
elif afweight is None: 
    afweight=1.
    print("Warning: acquisition function weight not defined. Use default: 1.")

gridtype=data_loaded.get("gridtype")
if gridtype=="continuous":
    igrid=0
    print(" * grid type: "+gridtype)
elif gridtype is None: 
    igrid=0
    print("Warning: grid type not defined. Use default: continuous")
else: raise ValueError("Error: "+gridtype+" not implemented.")

nlamvar=data_loaded.get("nlamvar")
if nlamvar is None: raise ValueError("Error: nlamvar not defined.")

maxlam=data_loaded.get("maxlam")
minlam=data_loaded.get("minlam")
if maxlam is None or minlam is None: raise ValueError("Error: maxlam and/or minlam not defined.")

mfunc=data_loaded.get("mfunc")
if mfunc=="Er":
    ifun=0
    print(" * minimization function: sum of weighted relative errors")
elif mfunc=="Er**2":
    ifun=1
    print(" * minimization function: sum of weighted square relative errors")
elif mfunc is None: 
    imin=0
    print("Warning: minimization function not defined. Use default: Er ")
else: raise ValueError("Error: "+mfunc+" not implemented.")

minst=data_loaded.get("minst")
if minst=="gr+ex":
    imin=0
    print(" * states included in minimization: ground + excited")
elif minst=="ex":
    imin=1
    print(" * states included in minimization: excited")
elif minst is None: 
    ifun=0
    print("Warning: states to be minimized not defined. Use default: ground + excited")
else: raise ValueError("Error: "+minst+" not implemented.")

nener=data_loaded.get("nener")
if nener is None: raise ValueError("Error: nener not defined.")

wi=data_loaded.get("wi")
if wi=="eq" or wi=="gi":
    if wi=="eq": igi=0
    if wi=="gi": igi=1
    weight=np.full(nener,1.)
    print(" * weight type: "+wi)
elif wi=="inp":
    inpweight=data_loaded.get("weight")
    linpw=len(inpweight)
    if linpw<nener: # fill the missing wi with the last input value
        for i in range(linpw,nener):
            inpweight.append(inpweight[linpw-1])
    weight=np.array(listweight)
    if inpweight is None or linpw==0:
        raise ValueError("Error: weight not defined.")
elif wi is None:
    igi=1
    print("Warning: wi not defined. Use default: eq")
else: raise ValueError("Error: "+wi+"not implemented.")

vpol=data_loaded.get("vpol")
if vpol==True: 
    ipol=1
    autovarlambda.polbck.ipol=ipol
    print(" * include polarization ON.")
else:
    print(" * no polarization.")

global ii,npolvar

# Define useful functions

def atom_name(nzion):
    nzion=abs(nzion)
    atomname=['H','He','Li','Be','B','C','N','O','F','Ne','Na','Mg']
    nname=len(atomname)
    if nzion>nname: raise ValueError("atomname not defined for Z="+str(nzion))
    for i in range(nname):
        if nzion==i+1: chatom=atomname[i] 
    return chatom

def pot_type(nzion):
    if nzion < 0:
        chtype="STO"
    else:
        chtype="TFDA"
    return chtype

# Define function for writing iteration number in a human-readable-way
def human_format(num):
    magnitude=0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000
    return '%.f%s' % (num, ['', 'K', 'M', 'G', 'T', 'P'][magnitude])

# Determine file name string vector from fortran subroutine data
def def_filename():
    global ncfg, filename,initer
    ncfg=autovarlambda.salgebbck.mxconf
    nzion=autovarlambda.sminimbck.nzion
    chatom=atom_name(nzion)
    chtype=pot_type(nzion)
    cfgs=str(ncfg)+"CFG"+chtype
    idrun=str(initer)+mtype+human_format(maxevals)
    filename=chatom+cfgs+"_"+idrun

# Open .out file
def open_fout():
    global fener
    fener=open(filename+".out","w")

# Define and initialize variables
def init_var():
    enere=np.zeros(nener)
    enerc=np.zeros(nener)
    neex=int()
    return enere, enerc, neex


############  SPACES  ############
def mylamspace(minlam,maxlam):
    space=[]
    for i in range(nlamvar):
        alam=str(i+1)
        space.append({'name': 'lam'+alam, 'type': gridtype, 'domain': (minlam,maxlam)})
    return space

def mypolspace(alfd_min,alfd_max,rcut_min,rcut_max,npolvar):
    space=[]
    for i in range(npolvar):
        apol=str(i)
        space.append({'name': 'alfd'+apol,'type': 'continuous', 'domain': (alfd_min,alfd_max)})
        space.append({'name': 'rcut'+apol,'type': 'continuous', 'domain': (rcut_min,rcut_max)})
    return space

############  KERNELS  ############
def mykernel(rbf,stdperiodic):
    if rbf==1: kernel=GPy.kern.RBF(input_dim=nlamvar,variance=varf,lengthscale=lf,ARD=ARD)
    if rbf==1 and stdperiodic==1:
        kernel1=GPy.kern.RBF(input_dim=nlamvar,variance=varf,lengthscale=lf,ARD=ARD)
        kernel2=GPy.kern.StdPeriodic(input_dim=nlamvar,variance=varf,period=None,lengthscale=None,ARD1=ARD,ARD2=ARD)
        kernel=kernel1*kernel2
    return kernel

def mypolkernel(rbf,stdperiodic):
    if rbf==1: kernel=GPy.kern.RBF(input_dim=2,variance=varf,lengthscale=lf,ARD=ARD)
    return kernel
###################################

def error_relat(valexact,valcomp,neex):
    if valexact == 0.0 or valcomp == 0.0:
        error=5./neex
    else:
        error=abs((valexact-valcomp)/valexact)*100
    return error

def loss_sumwE():
    global imin
    neex=autovarlambda.eei_ls.ne
    enere=autovarlambda.eicompare.enere
    enerc=autovarlambda.eicompare.enerc
    loss=0.0
    if igi==1: weight=autovarlambda.cei_ls.gic
    if igi==0: weight=autovarlambda.cei_ls.gic*0+1.
    for i in range(imin,neex): # consider ground energy and/or-only excited states
        erp=error_relat(enere[i],enerc[i],neex)
        if ifun==1: erp=erp*erp
        loss=loss + weight[i]*erp
#         print(' {} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f}'.format(i,enere[i],enerc[i],erp,weight[i],loss))
    return loss

def loss_sumaki():
    global imin
    ntrtot=autovarlambda.akicompare.ntrtot
    vakie=autovarlambda.akicompare.vakie
    vakic=autovarlambda.akicompare.vakic
    loss=0.0
    for i in range(imin,ntrtot): # consider a certain number of transitions
        erp=error_relat(vakie[i],vakic[i],ntrtot)
        if ifun==1: erp=erp*erp
        loss=loss+erp
#        print(' {} {:.3e} {:.3e} {:.4f} {:.4f}'.format(i,vakie[i],vakic[i],erp,loss))
    return loss
    
def loss_total(ii,x):
    global icost
    lx=len(x)
    lei=loss_sumwE()
    xp='{:4d}: [ '
    for i in range(lx):
        xp=xp+'{vp['+str(i)+']:.{prec}f} '
    xp=xp+'] // {l1:.{prec}f}'
    if icost==0:
        loss=lei
        print(xp.format(ii,vp=x,l1=lei,prec=4))
    if icost==1:
        laki=loss_sumaki()
        loss=lei+laki
        xp=xp+' + {l2:.{prec}f} = {lt:.{prec}f}'
        print(xp.format(ii,vp=x,l1=lei,l2=laki,lt=loss,prec=4))
    return loss

def check_errlog():
    ierr=0
    errlog=os.path.exists("errlog")
    if errlog == True: 
        ierr=1
        os.system("rm errlog")
    autovarlambda.errlogbck.ierr=ierr
    return

def py_runAS_lam(lam,nlamvar):
    check_errlog()
    autovarlambda.run_varlam(lam,nlamvar)
    check_errlog()
    autovarlambda.inp_comp()
    autovarlambda.compare_ei()
    autovarlambda.compare_aki()
    return

def py_runAS_pol(pol,npolvar):
    check_errlog()
    autovarlambda.run_varpol(pol,npolvar)
    check_errlog()
    autovarlambda.inp_comp()
    autovarlambda.compare_ei()
    autovarlambda.compare_aki()
    return

def var_lambda(x):
    global ii
    ii=ii+1
    lam=x[0,:]
    py_runAS_lam(lam,nlamvar)
    loss=loss_total(ii,lam)
    return loss

def var_pol(x):
    global ii,npolvar
    pol=x[0,:]
    ii=ii+1
    py_runAS_pol(pol,npolvar)
    loss=loss_total(ii,pol)
    return loss

def run_mybo_pol(space,kernel,initer,maxevals,varf,lf,noise_var,exact_feval,optimize_restarts,ARD,xi):
    #iseed=initer*123456
    iseed=50*123456
    seed(iseed)
    enere, enerc, neex=init_var()
    cpdas="cp das_" + str(cfgs) + "CFG das"
    os.system(cpdas)
    autovarlambda.open_files()
    autovarlambda.inp_das()
    autovarlambda.inp_obs()
    dummyne=autovarlambda.eei_ls.ne.copy()
    if nener != dummyne:
        autovarlambda.eei_ls.ne=nener
    def_filename()
    if ncfg != cfgs:
        print(" >>>> configuration mismatch !!!! ")
        sys.exit()
    open_fout()
    t0=time.time()
    global ii
    ii=0
    model=GPyOpt.models.GPModel(kernel=kernel,noise_var=noise_var,exact_feval=exact_feval,
                                optimizer='lbfgs',max_iters=1000,optimize_restarts=optimize_restarts,
                                verbose=False,ARD=ARD)
    dspace=GPyOpt.Design_space(space=space)
    objective=GPyOpt.core.task.SingleObjective(var_pol)
    initial_design=GPyOpt.experiment_design.initial_design('random',dspace,initer)
    acquisition_optimizer=GPyOpt.optimization.AcquisitionOptimizer(dspace, optimizer='lbfgs')
    acquisition=GPyOpt.acquisitions.AcquisitionEI(model,dspace,acquisition_optimizer,jitter=xi)
    evaluator=GPyOpt.core.evaluators.Sequential(acquisition)
    myBopt=GPyOpt.methods.ModularBayesianOptimization(model,dspace,objective,acquisition,
                                                      evaluator,X_init=initial_design,normalize_Y=True,
                                                      model_update_interval=1)
    myBopt.run_optimization(maxevals,eps=1e-7,verbosity=False)
    myBopt.plot_acquisition("polparam_"+str(initer)+"GP"+str(maxevals)+".eps")
    myBopt.save_evaluations(filename+".dat")
    t1=time.time()
    total=(t1-t0)/60.
    print_optpol(total,myBopt,filename,maxevals)
    return 

def print_optpol(total,myBopt,filename,maxevals):
    global fmin,weight,fener,npolvar
    pol_best=myBopt.x_opt
    fmin=myBopt.fx_opt
    py_runAS_pol(pol_best,npolvar)
    neex=autovarlambda.eicompare.neex
    enere=autovarlambda.eicompare.enere
    enerc=autovarlambda.eicompare.enerc
    loss_best=loss_sumwE()
    erelat=[]
    erroregr=error_relat(enere[0],enerc[0],neex)
    erelat.append(erroregr)
    xalpha='\n alpha = '
    xrcut='  rcut = '
    ii=0
    for i in range(npolvar):
        j=i+ii
        xalpha=xalpha+'{vp['+str(j)+']:.{prec}f} '
        xrcut=xrcut+'{vp['+str(j+1)+']:.{prec}f} '
        ii=ii+1
        
    print("\n Total time: {:.{prec}f} min".format(total,prec=4))
    print(xalpha.format(vp=pol_best,prec=4))
    print(xrcut.format(vp=pol_best,prec=4))
    print("\n Number of evaluations={:10d}".format(maxevals),file=fener)
    print("                  Time={:10.3f} min".format(total),file=fener)
    print("-"*80,file=fener)
    print(" Best results:",file=fener)
    print("-"*80,file=fener)
    print(xalpha.format(vp=pol_best,prec=4),file=fener)
    print(xrcut.format(vp=pol_best,prec=4),file=fener)
    print("\n  Ground State Energy ={:12.6f}".format(enerc[0]),
          "\n                  NIST={:12.6f}".format(enere[0]),
          "\n                   Er%={:12.4f} %".format(erroregr),file=fener)
    for i in range(1,neex):
        errorex=error_relat(enere[i],enerc[i],neex)
        erelat.append(errorex)
        print("\n       {} Excit. Energy={:12.6f}".format(i,enerc[i]),
              "\n                  NIST={:12.6f}".format(enere[i]),
              "\n                   Er%={:12.6f} %".format(errorex),file=fener)
    print("\n            Total loss={:12.4f} %".format(loss_best),file=fener)
    autovarlambda.print_ener()
    os.system("mv relat_error.dat "+filename+".erp")
    os.system("mv tmp "+filename+".das")
    fener.close()
    
    print("\n Ji:")
    for i in erelat:
        print('{:12.4f}'.format(i,))
    print("\n  Jtot = {:.{prec}f}".format(fmin,prec=4))
    
    return


===>>> Input parameters <<<===

 * initial evaluations: 20
 * evaluations: 50
 * model type: GP
 * acquisition function type: EI
 * acquisition function weight: 0.0001
 * grid type: continuous
 * minimization function: sum of weighted relative errors
 * states included in minimization: ground + excited
 * weight type: eq
 * include polarization ON.


In [2]:
alfd_min=0.001
alfd_max=0.100
rcut_min=0.50
rcut_max=1.50

In [4]:
noise_var=None
exact_feval=True
ARD=True
varf=2.5
lf=0.1
xi=0.0001
optimize_restarts=5

initer=300
maxevals=100
nener=10
npolvar=3
icost=0

space=mypolspace(alfd_min,alfd_max,rcut_min,rcut_max,npolvar)
kernel=mykernel(rbf=1,stdperiodic=0)
run_mybo_pol(space,kernel,initer,maxevals,varf,lf,noise_var,exact_feval,optimize_restarts,ARD,xi)

   1: [ 0.0403 1.1870 0.0597 0.5022 0.0395 1.1261 ] // 4.6408
   2: [ 0.0974 0.5997 0.0365 1.3770 0.0719 0.5438 ] // 11.8166
   3: [ 0.0919 0.6134 0.0301 0.8912 0.0264 0.8246 ] // 9.3609
   4: [ 0.0188 1.4310 0.0210 1.0828 0.0396 0.9773 ] // 5.1253
   5: [ 0.0777 0.7441 0.0610 0.9234 0.0718 1.1627 ] // 0.9542
   6: [ 0.0699 1.3141 0.0972 0.6800 0.0854 1.0940 ] // 4.2613
   7: [ 0.0259 1.2172 0.0328 1.3050 0.0518 1.4486 ] // 5.0100
   8: [ 0.0353 0.9919 0.0809 0.9866 0.0456 1.1125 ] // 4.4735
   9: [ 0.0987 1.1096 0.0059 1.0200 0.0534 1.4621 ] // 4.1866
  10: [ 0.0726 0.8354 0.0994 1.2967 0.0708 1.3166 ] // 2.3421
  11: [ 0.0345 1.0656 0.0746 1.1199 0.0132 1.1566 ] // 4.6904
  12: [ 0.0534 1.3157 0.0684 1.4460 0.0109 1.1817 ] // 4.8984
  13: [ 0.0660 1.3044 0.0609 1.0569 0.0558 1.0598 ] // 4.7838
  14: [ 0.0062 0.8049 0.0499 1.0272 0.0455 1.1353 ] // 4.8609
  15: [ 0.0719 0.5824 0.0371 1.3291 0.0057 0.6549 ] // 9.0730
  16: [ 0.0132 1.4723 0.0590 1.4114 0.0471 0.5582 ] // 5.1309
  17: [

 143: [ 0.0542 0.5405 0.0638 0.6373 0.0818 1.0175 ] // 9.8048
 144: [ 0.0070 0.5302 0.0667 1.3246 0.0102 1.4605 ] // 3.0870
 145: [ 0.0217 1.2021 0.0127 1.4408 0.0499 1.0328 ] // 5.0543
 146: [ 0.0371 0.7037 0.0567 1.3143 0.0734 0.9257 ] // 2.0183
 147: [ 0.0882 0.6466 0.0420 0.9331 0.0293 0.8793 ] // 5.8709
 148: [ 0.0593 0.5957 0.0184 0.8848 0.0948 0.9258 ] // 5.4663
 149: [ 0.0937 0.8825 0.0474 0.9123 0.0767 0.6454 ] // 2.3523
 150: [ 0.0797 0.5648 0.0639 1.1181 0.0847 1.4246 ] // 12.9261
 151: [ 0.0501 0.5244 0.0809 1.2847 0.0762 0.9323 ] // 10.4113
 152: [ 0.0110 0.9348 0.0572 1.4989 0.0070 0.8383 ] // 4.9325
 153: [ 0.0255 1.2750 0.0155 0.5463 0.0825 0.9439 ] // 5.0157
 154: [ 0.0624 0.5656 0.0136 0.8428 0.0174 1.1575 ] // 8.8528
 155: [ 0.0327 1.4867 0.0635 1.4409 0.0252 0.7369 ] // 5.0703
 156: [ 0.0205 0.5508 0.0224 0.7884 0.0721 1.2435 ] // 0.9466
 157: [ 0.0472 0.7424 0.0883 0.9047 0.0980 1.2280 ] // 1.9132
 158: [ 0.0011 0.7202 0.0358 1.1895 0.0376 1.2225 ] // 5.0962
 159: 

 280: [ 0.0543 0.6722 0.0658 0.6652 0.0122 0.7188 ] // 1.3269
 281: [ 0.0804 1.2793 0.0430 0.7121 0.0132 1.0355 ] // 4.6423
 282: [ 0.0882 1.4733 0.0020 0.6180 0.0791 1.4546 ] // 4.9329
 283: [ 0.0218 1.2091 0.0940 0.7554 0.0660 0.9807 ] // 4.8137
 284: [ 0.0164 0.7071 0.0790 0.5609 0.0124 0.5727 ] // 3.2460
 285: [ 0.0782 1.4796 0.0703 0.9767 0.0656 0.8491 ] // 4.8499
 286: [ 0.0209 0.5856 0.0706 1.4599 0.0657 0.8813 ] // 1.1516
 287: [ 0.0637 1.2359 0.0124 1.1091 0.0018 0.8131 ] // 4.7896
 288: [ 0.0119 1.3429 0.0241 0.9371 0.0336 1.4129 ] // 5.1229
 289: [ 0.0944 1.3411 0.0405 0.7438 0.0805 1.3015 ] // 4.6711
 290: [ 0.0874 1.0568 0.0102 0.5144 0.0820 1.3803 ] // 4.0500
 291: [ 0.0049 0.9267 0.0840 0.9711 0.0286 1.1867 ] // 4.9571
 292: [ 0.0665 0.8777 0.0324 1.3896 0.0925 0.9153 ] // 3.1756
 293: [ 0.0760 0.6228 0.0692 1.2878 0.0814 0.5610 ] // 6.0693
 294: [ 0.0152 1.2737 0.0388 1.3321 0.0127 0.5102 ] // 5.0986
 295: [ 0.0803 1.3508 0.0621 1.1396 0.0932 0.5521 ] // 4.7754
 296: [ 

In [6]:
noise_var=None
exact_feval=True
ARD=True
varf=2.5
lf=0.1
xi=0.0001
optimize_restarts=5

initer=300
maxevals=100
nener=10
npolvar=3
icost=0

space=mypolspace(alfd_min,alfd_max,rcut_min,rcut_max,npolvar)
kernel=mykernel(rbf=1,stdperiodic=0)
run_mybo_pol(space,kernel,initer,maxevals,varf,lf,noise_var,exact_feval,optimize_restarts,ARD,xi)

   1: [ 0.0403 1.1870 0.0597 0.5022 0.0395 1.1261 ] // 4.6408
   2: [ 0.0974 0.5997 0.0365 1.3770 0.0719 0.5438 ] // 11.8166
   3: [ 0.0919 0.6134 0.0301 0.8912 0.0264 0.8246 ] // 9.3609
   4: [ 0.0188 1.4310 0.0210 1.0828 0.0396 0.9773 ] // 5.1253
   5: [ 0.0777 0.7441 0.0610 0.9234 0.0718 1.1627 ] // 0.9542
   6: [ 0.0699 1.3141 0.0972 0.6800 0.0854 1.0940 ] // 4.2613
   7: [ 0.0259 1.2172 0.0328 1.3050 0.0518 1.4486 ] // 5.0100
   8: [ 0.0353 0.9919 0.0809 0.9866 0.0456 1.1125 ] // 4.4735
   9: [ 0.0987 1.1096 0.0059 1.0200 0.0534 1.4621 ] // 4.1866
  10: [ 0.0726 0.8354 0.0994 1.2967 0.0708 1.3166 ] // 2.3421
  11: [ 0.0345 1.0656 0.0746 1.1199 0.0132 1.1566 ] // 4.6904
  12: [ 0.0534 1.3157 0.0684 1.4460 0.0109 1.1817 ] // 4.8984
  13: [ 0.0660 1.3044 0.0609 1.0569 0.0558 1.0598 ] // 4.7838
  14: [ 0.0062 0.8049 0.0499 1.0272 0.0455 1.1353 ] // 4.8609
  15: [ 0.0719 0.5824 0.0371 1.3291 0.0057 0.6549 ] // 9.0730
  16: [ 0.0132 1.4723 0.0590 1.4114 0.0471 0.5582 ] // 5.1309
  17: [

 134: [ 0.0399 0.9433 0.0867 1.1721 0.0654 0.9629 ] // 4.2471
 135: [ 0.0867 1.1007 0.0381 1.1319 0.0874 1.4546 ] // 4.2365
 136: [ 0.0691 1.0637 0.0114 1.1492 0.0929 0.6005 ] // 4.3415
 137: [ 0.0217 0.8232 0.0421 0.9002 0.0199 0.8715 ] // 4.2494
 138: [ 0.0384 0.6686 0.0704 1.3653 0.0253 1.2859 ] // 1.1187
 139: [ 0.0266 1.2129 0.0438 1.1190 0.0445 1.2575 ] // 4.9747
 140: [ 0.0882 0.9364 0.0280 1.4804 0.0390 1.4598 ] // 3.2197
 141: [ 0.0054 1.0979 0.0418 1.4470 0.0434 0.9216 ] // 5.1315
 142: [ 0.0135 0.7503 0.0795 0.8724 0.0103 1.0164 ] // 4.2006
 143: [ 0.0542 0.5405 0.0638 0.6373 0.0818 1.0175 ] // 9.8048
 144: [ 0.0070 0.5302 0.0667 1.3246 0.0102 1.4605 ] // 3.0870
 145: [ 0.0217 1.2021 0.0127 1.4408 0.0499 1.0328 ] // 5.0543
 146: [ 0.0371 0.7037 0.0567 1.3143 0.0734 0.9257 ] // 2.0183
 147: [ 0.0882 0.6466 0.0420 0.9331 0.0293 0.8793 ] // 5.8709
 148: [ 0.0593 0.5957 0.0184 0.8848 0.0948 0.9258 ] // 5.4663
 149: [ 0.0937 0.8825 0.0474 0.9123 0.0767 0.6454 ] // 2.3523
 150: [ 

 269: [ 0.0842 0.7204 0.0531 0.9036 0.0572 1.0157 ] // 1.7462
 270: [ 0.0749 0.9192 0.0502 1.4782 0.0890 0.6576 ] // 3.3507
 271: [ 0.0982 0.8322 0.0729 0.7129 0.0841 1.3091 ] // 1.2476
 272: [ 0.0232 1.2747 0.0923 1.0954 0.0709 0.6531 ] // 4.9566
 273: [ 0.0691 0.8382 0.0623 0.9687 0.0366 1.1839 ] // 2.5073
 274: [ 0.0535 0.9134 0.0315 0.9381 0.0483 1.3584 ] // 3.8148
 275: [ 0.0537 1.4376 0.0819 1.1076 0.0601 1.4274 ] // 4.9168
 276: [ 0.0753 1.1540 0.0694 1.1265 0.0355 1.1176 ] // 4.4676
 277: [ 0.0676 1.3248 0.0799 0.9444 0.0246 1.2569 ] // 4.7397
 278: [ 0.0037 0.6799 0.0199 1.2002 0.0433 0.9733 ] // 4.8330
 279: [ 0.0279 1.0419 0.0468 1.2329 0.0943 1.1791 ] // 4.7861
 280: [ 0.0543 0.6722 0.0658 0.6652 0.0122 0.7188 ] // 1.3269
 281: [ 0.0804 1.2793 0.0430 0.7121 0.0132 1.0355 ] // 4.6423
 282: [ 0.0882 1.4733 0.0020 0.6180 0.0791 1.4546 ] // 4.9329
 283: [ 0.0218 1.2091 0.0940 0.7554 0.0660 0.9807 ] // 4.8137
 284: [ 0.0164 0.7071 0.0790 0.5609 0.0124 0.5727 ] // 3.2460
 285: [ 

In [4]:
alfd_min=0.0005
alfd_max=0.5000
rcut_min=0.1000
rcut_max=1.9000

In [5]:
noise_var=None
exact_feval=True
ARD=True
varf=2.5
lf=0.1
xi=0.0001
optimize_restarts=5

initer=400
maxevals=400
nener=10
npolvar=3
icost=0

space=mypolspace(alfd_min,alfd_max,rcut_min,rcut_max,npolvar)
kernel=mykernel(rbf=1,stdperiodic=0)
run_mybo_pol(space,kernel,initer,maxevals,varf,lf,noise_var,exact_feval,optimize_restarts,ARD,xi)

   1: [ 0.1989 1.2512 0.0385 0.7999 0.4660 1.1645 ] // 3.8901
   2: [ 0.4870 0.9200 0.4630 1.3896 0.2085 0.6562 ] // 7.2024
   3: [ 0.4591 0.3374 0.0734 0.5625 0.4137 1.8198 ] // 844.3874
   4: [ 0.0904 0.2637 0.1574 0.8012 0.2305 0.8524 ] // 363.7509
   5: [ 0.3872 1.0303 0.2595 1.3873 0.0781 1.5531 ] // 1.4317
   6: [ 0.3484 0.5458 0.4346 1.6352 0.0923 1.6112 ] // 87.3046
   7: [ 0.1261 1.6510 0.0447 1.0239 0.1158 0.4556 ] // 4.8872
   8: [ 0.1733 1.2275 0.0188 0.9101 0.0328 1.2799 ] // 4.0169
   9: [ 0.4933 0.1454 0.3272 1.0526 0.2267 1.4154 ] // 30520.8146
  10: [ 0.3619 0.3252 0.4756 1.3685 0.0852 0.6884 ] // 754.1085
  11: [ 0.1696 0.8734 0.4399 0.3213 0.4582 0.9415 ] // 92.8747
  12: [ 0.2646 1.6819 0.0891 0.2801 0.3030 1.7039 ] // 3.5969
  13: [ 0.3287 1.6184 0.3800 1.0963 0.1306 1.8935 ] // 3.6176
  14: [ 0.0265 0.5145 0.4194 0.9095 0.1302 1.4712 ] // 4.8936
  15: [ 0.3581 1.6051 0.2235 0.1851 0.1710 1.7286 ] // 123.9671
  16: [ 0.0622 0.8151 0.3996 0.9383 0.0839 0.4719 ] // 1

 136: [ 0.3441 0.2749 0.4309 1.7700 0.4574 0.7634 ] // 1371.1166
 137: [ 0.1052 0.9905 0.1616 0.4444 0.3373 0.1684 ] // 2.2833
 138: [ 0.1890 0.6039 0.2921 0.5423 0.0153 1.7610 ] // 28.6883
 139: [ 0.1298 1.7534 0.0354 0.8916 0.3553 1.5973 ] // 4.9347
 140: [ 0.4406 1.0844 0.2603 0.7902 0.2103 0.4095 ] // 1.2836
 141: [ 0.0227 0.5376 0.1802 0.8710 0.1073 0.1111 ] // 1.8117
 142: [ 0.0633 0.1527 0.3639 0.2697 0.1508 1.2968 ] // 1582.2204
 143: [ 0.2687 1.6278 0.2746 1.5691 0.3353 1.6653 ] // 4.4459
 144: [ 0.0309 0.6165 0.0279 0.2674 0.0462 0.8181 ] // 0.9142
 145: [ 0.1051 0.9411 0.3298 0.9890 0.2370 0.6759 ] // 2.0173
 146: [ 0.1829 0.6592 0.4936 1.4160 0.0809 1.0299 ] // 16.1974
 147: [ 0.4402 0.3901 0.3120 0.6143 0.2343 1.5753 ] // 456.5412
 148: [ 0.2945 1.5763 0.2816 1.8050 0.1963 1.5736 ] // 4.3572
 149: [ 0.4683 1.1257 0.4685 1.4762 0.3201 0.3690 ] // 0.9523
 150: [ 0.3975 0.9518 0.0205 1.6212 0.2628 0.3930 ] // 3.3398
 151: [ 0.2484 1.2486 0.0229 1.4681 0.2020 0.7375 ] // 3.644

 266: [ 0.2057 0.7769 0.3203 0.2666 0.3383 0.3241 ] // 24.7726
 267: [ 0.4352 0.9311 0.0883 1.5544 0.1966 1.3431 ] // 4.9590
 268: [ 0.1203 0.6664 0.3114 0.8838 0.1640 1.5192 ] // 8.8865
 269: [ 0.4203 0.9561 0.4597 1.1215 0.2289 0.2227 ] // 4.1001
 270: [ 0.3733 1.5941 0.4734 1.6991 0.0147 0.5519 ] // 4.0687
 271: [ 0.4910 0.8327 0.3865 1.6105 0.4239 1.2395 ] // 14.2969
 272: [ 0.1126 1.3068 0.0724 1.3707 0.3888 1.5022 ] // 4.5723
 273: [ 0.3443 1.7444 0.2449 0.7477 0.2917 0.2389 ] // 3.6251
 274: [ 0.2656 0.3041 0.2607 0.9600 0.0918 1.0353 ] // 697.6887
 275: [ 0.2662 1.4834 0.4598 1.1737 0.4442 1.1463 ] // 3.4839
 276: [ 0.3752 1.2614 0.3804 0.7279 0.4347 0.8012 ] // 1.8297
 277: [ 0.3367 0.2501 0.0272 0.5295 0.4288 0.9391 ] // 1942.3198
 278: [ 0.0141 0.3079 0.3115 0.8698 0.2529 0.7711 ] // 28.0421
 279: [ 0.1361 0.1490 0.3185 1.7958 0.0246 1.8568 ] // 5338.5234
 280: [ 0.2692 0.6213 0.1557 0.3038 0.1412 1.4804 ] // 37.1250
 281: [ 0.4009 1.6804 0.0577 0.3210 0.4418 0.8882 ] // 4.0

 401: [ 0.0335 0.1984 0.0005 0.1000 0.5000 1.9000 ] // 316.6507
 402: [ 0.0005 0.3028 0.5000 1.9000 0.5000 0.1043 ] // 3.8239
 403: [ 0.5000 0.8981 0.0005 0.1000 0.0005 1.0968 ] // 8.5562
 404: [ 0.0005 0.1000 0.0005 0.1000 0.5000 1.5092 ] // 16.3559
 405: [ 0.1497 0.3576 0.5000 0.1000 0.5000 0.6755 ] // 209.0072
 406: [ 0.0005 1.9000 0.5000 0.1000 0.5000 1.5668 ] // 26.2923
 407: [ 0.1530 1.0515 0.0005 0.1000 0.0005 0.4550 ] // 3.1921
 408: [ 0.5000 0.4555 0.5000 0.1000 0.5000 1.2534 ] // 273.7420
 409: [ 0.2188 0.2025 0.0005 0.1000 0.5000 1.1245 ] // 2720.0326
 410: [ 0.0005 0.2429 0.0005 1.9000 0.3790 1.9000 ] // 2.8369
 411: [ 0.0005 0.1522 0.0005 0.1000 0.0005 0.1000 ] // 3.1951
 412: [ 0.2666 0.1806 0.5000 0.1000 0.5000 1.9000 ] // 5566.4866
 413: [ 0.0005 0.1000 0.0005 1.9000 0.2841 0.1000 ] // 16.3506
 414: [ 0.0005 0.1782 0.5000 0.1000 0.3668 1.9000 ] // 27.5746
 415: [ 0.2081 0.1983 0.5000 0.1000 0.3119 1.9000 ] // 2797.1660
 416: [ 0.5000 1.9000 0.0005 1.9000 0.2118 1.6269 ]

KeyboardInterrupt: 